In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np # For saving data

In [2]:
import mediapipe as mp
mp_holistic = mp.solutions.holistic


In [3]:
mp_holistic = mp.solutions.holistic
holistic_detector = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=2,              # more accurate for detailed hand tracking
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [4]:
video_folder = 'videos'
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

In [7]:
landmark_folder = 'landmarks'

# Create landmark folder if it doesn't exist
if not os.path.exists(landmark_folder):
    os.makedirs(landmark_folder)

# List all video files
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

print(f"Starting holistic landmark extraction for videos in '{video_folder}'...")
print(f"Saving landmarks to '{landmark_folder}'...\n")

# --- Loop Through Videos ---
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_id = os.path.splitext(video_file)[0]
    save_landmark_path = os.path.join(landmark_folder, f"{video_id}.npy")

    # --- Skip if already extracted ---
    if os.path.exists(save_landmark_path):
        print(f"Skipping {video_file} - .npy file already exists.")
        continue

    print(f"Processing {video_file}...")
    cap = cv2.VideoCapture(video_path)

    # --- Check if video opened successfully ---
    if not cap.isOpened():
        print(f"⚠️  Warning: Could not open video file {video_file}. Skipping.")
        cap.release()
        continue

    frame_landmarks_list = []
    processed_successfully = True

    try:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break  # End of video or read error

            # Convert to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False

            # Process with holistic
            results = holistic_detector.process(image_rgb)
            image_rgb.flags.writeable = True

            # --- Collect pose + hand landmarks ---
            landmarks_current_frame = []

            # Pose landmarks (33)
            if results.pose_landmarks:
                for lm in results.pose_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, lm.visibility])
            else:
                landmarks_current_frame.extend([0.0] * 33 * 4)

            # Left hand landmarks (21)
            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, 1.0])  # visibility=1.0 since not provided
            else:
                landmarks_current_frame.extend([0.0] * 21 * 4)

            # Right hand landmarks (21)
            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, 1.0])
            else:
                landmarks_current_frame.extend([0.0] * 21 * 4)

            frame_landmarks_list.append(landmarks_current_frame)

    except Exception as e:
        print(f"⚠️  Error processing {video_file}: {e}")
        processed_successfully = False

    finally:
        cap.release()

    # --- Save landmarks ---
    if processed_successfully and frame_landmarks_list:
       video_landmark_data = np.array(frame_landmarks_list, dtype=np.float32)
       np.save(save_landmark_path, video_landmark_data)

       print(f"✅ Saved landmarks for {video_file}")
    else:
        print(f"⚠️  Skipped saving for {video_file} due to errors or no frames.\n")

# Close holistic detector
holistic_detector.close()

print("\n✅ Holistic landmark extraction complete!")

Starting holistic landmark extraction for videos in 'videos'...
Saving landmarks to 'landmarks'...

Processing 01457.mp4...
⚠️  Error processing 01457.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 01457.mp4 due to errors or no frames.

Processing 01467.mp4...
⚠️  Error processing 01467.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 01467.mp4 due to errors or no frames.

Processing 01468.mp4...
⚠️  Error processing 01468.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 01468.mp4 due to errors or no frames.

Processing 01469.mp4...
⚠️  Error processing 01469.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 01469.mp4 due to errors or no frames.

Processing 05296.mp4...
⚠️  Error processing 05296.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 05296.mp4 due to errors or no frames.

Processing 05304.mp4...
⚠️  Error processing 05304.mp4: _graph is None in SolutionBase
⚠️  Skipped saving for 05304.mp4 due to errors or no frames.

Proces

ValueError: Closing SolutionBase._graph which is already None

In [5]:
import os
import cv2
import numpy as np
import mediapipe as mp

# --- Paths ---
video_folder = 'videos'      # <-- change this
landmark_folder = 'landmarks'      # <-- change this

os.makedirs(landmark_folder, exist_ok=True)

# Initialize MediaPipe holistic
mp_holistic = mp.solutions.holistic

# List all video files
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

print(f"Starting holistic landmark extraction for videos in '{video_folder}'...")
print(f"Saving landmarks to '{landmark_folder}'...\n")

# --- Loop Through Videos ---
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_id = os.path.splitext(video_file)[0]
    save_landmark_path = os.path.join(landmark_folder, f"{video_id}.npy")

    # Skip if already extracted
    if os.path.exists(save_landmark_path):
        print(f"Skipping {video_file} - .npy file already exists.")
        continue

    print(f"Processing {video_file}...")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"⚠️  Warning: Could not open video file {video_file}. Skipping.")
        cap.release()
        continue

    # Create a fresh holistic detector per video
    holistic_detector = mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=2,               # better finger tracking
        smooth_landmarks=True,
        enable_segmentation=False,
        refine_face_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    frame_landmarks_list = []
    processed_successfully = True

    try:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break  # End of video or error

            # Convert to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb.flags.writeable = False

            # Process with holistic
            results = holistic_detector.process(image_rgb)
            image_rgb.flags.writeable = True

            # --- Collect pose + hand landmarks ---
            landmarks_current_frame = []

            # Pose landmarks (33)
            if results.pose_landmarks:
                for lm in results.pose_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, lm.visibility])
            else:
                landmarks_current_frame.extend([0.0] * 33 * 4)

            # Left hand landmarks (21)
            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, 1.0])
            else:
                landmarks_current_frame.extend([0.0] * 21 * 4)

            # Right hand landmarks (21)
            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    landmarks_current_frame.extend([lm.x, lm.y, lm.z, 1.0])
            else:
                landmarks_current_frame.extend([0.0] * 21 * 4)

            frame_landmarks_list.append(landmarks_current_frame)

    except Exception as e:
        print(f"⚠️  Error processing {video_file}: {e}")
        processed_successfully = False

    finally:
        cap.release()
        holistic_detector.close()  # Close per video

    # --- Save landmarks ---
    if processed_successfully and frame_landmarks_list:
        video_landmark_data = np.array(frame_landmarks_list, dtype=np.float32)
        np.save(save_landmark_path, video_landmark_data)
        print(f"✅ Saved landmarks for {video_file}\n")
    else:
        print(f"⚠️  Skipped saving for {video_file} due to errors or no frames.\n")

print("\n✅ Holistic landmark extraction complete!")


Starting holistic landmark extraction for videos in 'videos'...
Saving landmarks to 'landmarks'...

Processing 01457.mp4...
✅ Saved landmarks for 01457.mp4

Processing 01467.mp4...
✅ Saved landmarks for 01467.mp4

Processing 01468.mp4...
✅ Saved landmarks for 01468.mp4

Processing 01469.mp4...
✅ Saved landmarks for 01469.mp4

Processing 05296.mp4...
✅ Saved landmarks for 05296.mp4

Processing 05304.mp4...
✅ Saved landmarks for 05304.mp4

Processing 05305.mp4...
✅ Saved landmarks for 05305.mp4

Processing 05307.mp4...
✅ Saved landmarks for 05307.mp4

Processing 06176.mp4...
✅ Saved landmarks for 06176.mp4

Processing 08608.mp4...
✅ Saved landmarks for 08608.mp4

Processing 08935.mp4...
✅ Saved landmarks for 08935.mp4

Processing 08936.mp4...
✅ Saved landmarks for 08936.mp4

Processing 08937.mp4...
✅ Saved landmarks for 08937.mp4

Processing 08938.mp4...
✅ Saved landmarks for 08938.mp4

Processing 08942.mp4...
✅ Saved landmarks for 08942.mp4

Processing 08944.mp4...
✅ Saved landmarks for